In [1]:
import load_data
import feature_attender
from sklearn import linear_model
import tensorflow as tf
from sklearn.metrics import mean_absolute_error
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth =True

set_session(tf.Session(config=config)) 


'''select  grp(BD orHC)
feature_types is the input features you want
B_1 is Dass-21 
B_2 is Altman
C is daily valence
E is daily 7 emotions
'''
dir_grp = 'HC'
feature_types = ['B_1','B_2','C','D','E']
# get attenders what u  want
attenders = feature_attender.get_wanna_attender(dir_grp,feature_types)

print(dir_grp,'attenders:',len(attenders))
# get input feature and scale (target_scale = 'ymrs' or 'hamd')
# each scale label is 4 dimension (3 factor of YMRS and 1 total score of YMRS)
x, label = load_data.loading_data(dir_grp,attenders,feature_types,target_scale='ymrs')
print('res', x.shape, label.shape)



Using TensorFlow backend.


Counting the type B_1
attender size: (222, 1)
-----------------------------------------------
Counting the type B_2
attender size: (221, 1)
-----------------------------------------------
Counting the type C
attender size: (292, 1)
-----------------------------------------------
Counting the type D
attender size: (266, 1)
-----------------------------------------------
Counting the type E
attender size: (250, 1)
-----------------------------------------------
HC attenders: 151
max (56,)
min (56,)
res: (151, 56)
x: (151, 56)
label: (151, 4)
res (151, 56) (151, 4)


In [2]:
import keras
from keras.models import Model
from sklearn import linear_model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.linear_model import LassoCV
from keras.utils import plot_model

extract factors

In [3]:
def neg_zero( L):
    for i in range( len(L)):
        if L[i] < 0:
            L[i] = 0
    return L

In [4]:
def get_alpha(x_,h_):
    global alpha_
    global check
    x_train = np.nan_to_num( x_)
    y_train = h_
    lassoCV = LassoCV(cv=10)
    lassoCV.fit( x_train, y_train)
    alpha_ = lassoCV.alpha_
    return alpha_

def Lasso_Regression_alpha(X_train,target):
    zero_flag = True
    alpha=get_alpha(X_train,target)
    model = linear_model.Lasso(alpha=alpha)
    model.fit(X_train,target)
    for i in model.coef_:
        if i != 0:
            zero_flag=False
            break
    if zero_flag == True:
        print('All zero coef!')
        model = linear_model.Lasso(alpha=0.1)
        model.fit(X_train,target)
    return model.coef_

In [5]:
import keras
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten

def Uni_NN(X_train,target,testX, testy,epoch,hidden_layers=1):
    inputs = Input(shape=X_train[0].shape, name='main_input')
    main_branch = Dense(200)(inputs)
    main_branch = Activation("relu")(main_branch)
    #NN layer
    for _ in range(hidden_layers):
        main_branch = Dense(200, activation='relu')(main_branch)
    #res
    main_branch = Dense(100, activation='relu')(main_branch)
    res = Dense(1, activation='relu', name='res')(main_branch)
    model = Model(inputs = inputs,
         outputs = [res])
    #model.summary()
    opt = keras.optimizers.Adam(lr=0.01)
    model.compile(optimizer=opt,
              loss={'res': 'mean_absolute_error'},
              loss_weights={'res': 1})
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
    model.fit({'main_input': X_train},
              {'res': target},
              epochs=epoch, batch_size=4,
              verbose=0,validation_data=(testX, testy),callbacks=[es]
             )
    print('fininsh')
    return model

5-fold 

In [6]:

    
def get_new_X(x,beta,index):
    res = []
    for i in x:
        tmp_res = []
        tmp = i*beta
        for idx in index:
            tmp_res.append(tmp[idx])
        tmp_res = np.array(tmp_res)
        res.append(tmp_res)
    return np.array(res)
        

In [7]:
import numpy as np
def get_coef_of_Lasso(x,h):
    Non_zero_index = []
    beta = np.zeros(74)
    # get beta of each factor

    #train
    #get core alpha
    beta = Lasso_Regression_alpha(x,h)
    for f in range(len(beta)):
        if beta[f] != 0:
            if f not in Non_zero_index:
                Non_zero_index.append(f)
                beta[f] = beta[f]
    #print('core_Non_zero_index',len(core_Non_zero_index),core_Non_zero_index)
    
    return  Non_zero_index , beta

In [8]:
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping
kf = KFold(n_splits=5, random_state=None, shuffle=False)

pm_mae = []
im_mae = []
em_mae = []
ts_mae = []

for train_index, test_index in kf.split(x):
    # 5-fold split test/train dataset
    X_test =  x[test_index]
    y_test =  label[test_index]
    
    X_train =  x[train_index]
    y_train =  label[train_index]

    print('X train:',X_train.shape)
    print('X test:',X_test.shape)
    print('Y train:',y_train.shape)
    print('Y test:',y_test.shape)
    #
    epoch = 5
    #epoch = 3
    #get_coef
    pm_Non_zero_index , pm_beta = get_coef_of_Lasso(X_train,y_train[:,0])
    print('pm_Non_zero_index',len(pm_Non_zero_index),pm_Non_zero_index)
    im_Non_zero_index , im_beta = get_coef_of_Lasso(X_train,y_train[:,1])
    print('im_Non_zero_index',len(im_Non_zero_index),im_Non_zero_index)
    em_Non_zero_index , em_beta = get_coef_of_Lasso(X_train,y_train[:,2])
    print('em_Non_zero_index',len(em_Non_zero_index),em_Non_zero_index)
    ts_Non_zero_index , ts_beta = get_coef_of_Lasso(X_train,y_train[:,3])
    print('ts_Non_zero_index',len(ts_Non_zero_index),ts_Non_zero_index)
    #pm
    if len(pm_Non_zero_index)!=0:
        pm_train = y_train[:,0]
        pm_X_train = get_new_X(X_train,pm_beta,pm_Non_zero_index)
        pm_test = y_test[:,0]
        pm_X_test = get_new_X(X_test,pm_beta,pm_Non_zero_index)
        model1 = Uni_NN(pm_X_train,pm_train,pm_X_test,pm_test,epoch )
        pm_rmae = mean_absolute_error( pm_test, model1.predict( pm_X_test))
        pm_mae.append(pm_rmae)
    else:
        pm_test = y_test[:,0]
        pm_rmae = mean_absolute_error( pm_test, np.zeros(len(pm_test)))
        pm_mae.append(pm_rmae)
    #im
    if len(im_Non_zero_index)!=0:
        im_train = y_train[:,1]
        im_X_train = get_new_X(X_train,im_beta,im_Non_zero_index)
        im_test = y_test[:,1]
        im_X_test = get_new_X(X_test,im_beta,im_Non_zero_index)
        model2 = Uni_NN(im_X_train,im_train,im_X_test,im_test,epoch )
        im_rmae = mean_absolute_error( im_test, model2.predict( im_X_test))
        im_mae.append(im_rmae)
    else:
        im_test = y_test[:,1]
        im_rmae = mean_absolute_error( im_test, np.zeros(len(im_test)))
        im_mae.append(im_rmae)
    #em
    if len(em_Non_zero_index)!=0:
        em_train = y_train[:,2]
        em_X_train = get_new_X(X_train,em_beta,em_Non_zero_index)
        em_test = y_test[:,2]
        em_X_test = get_new_X(X_test,em_beta,em_Non_zero_index)
        model3 = Uni_NN(em_X_train,em_train,em_X_test,em_test,epoch )
        em_rmae = mean_absolute_error( em_test, model3.predict( em_X_test))
        em_mae.append(em_rmae)
    else:
        em_test = y_test[:,3]
        em_rmae = mean_absolute_error( em_test, np.zeros(len(em_test)))
        em_mae.append(em_rmae)
    #ts
    if len(ts_Non_zero_index)!=0:
        ts_train = y_train[:,3]
        ts_X_train = get_new_X(X_train,ts_beta,ts_Non_zero_index)
        ts_test = y_test[:,3]
        ts_X_test = get_new_X(X_test,ts_beta,ts_Non_zero_index)
        model4 = Uni_NN(ts_X_train,ts_train,ts_X_test,ts_test,epoch )
        ts_rmae = mean_absolute_error( ts_test, model4.predict( ts_X_test))
        ts_mae.append(ts_rmae)
    else:
        ts_test = y_test[:,3]
        ts_rmae = mean_absolute_error( ts_test, np.zeros(len(ts_test)))
        ts_mae.append(ts_rmae)
#show result
pm_mean = sum(pm_mae) / len(pm_mae)
pm_std = np.std(pm_mae)
print('pm: %.2f ± %.2f' % (pm_mean, pm_std))
im_mean = sum(im_mae) / len(im_mae)
im_std = np.std(im_mae)
print('im: %.2f ± %.2f' % (im_mean, im_std))
em_mean = sum(em_mae) / len(em_mae)
em_std = np.std(em_mae)
print('em: %.2f ± %.2f' % (em_mean, em_std))
ts_mean = sum(ts_mae) / len(ts_mae)
ts_std = np.std(ts_mae)
print('ts: %.2f ± %.2f' % (ts_mean, ts_std))
    

X train: (120, 56)
X test: (31, 56)
Y train: (120, 4)
Y test: (31, 4)
All zero coef!
pm_Non_zero_index 0 []
im_Non_zero_index 3 [14, 37, 45]
em_Non_zero_index 1 [24]
ts_Non_zero_index 7 [0, 14, 24, 28, 33, 38, 53]
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 00002: early stopping
fininsh
Epoch 00002: early stopping
fininsh
Epoch 00002: early stopping
fininsh
X train: (121, 56)
X test: (30, 56)
Y train: (121, 4)
Y test: (30, 4)
All zero coef!
pm_Non_zero_index 0 []
im_Non_zero_index 3 [7, 14, 45]
All zero coef!
em_Non_zero_index 0 []
ts_Non_zero_index 6 [10, 14, 24, 33, 45, 53]
Epoch 00002: early stopping
fininsh
Epoch 00002: early stopping
fininsh
X train: (121, 56)
X test: (30, 56)
Y train: (121, 4)
Y test: (30, 4)
pm_Non_zero_index 5 [5, 40, 51, 53, 54]
im_Non_zero_index 4 [0, 14, 45, 53]
em_Non_zero_index 1 [7]
ts_Non_zero_index 4 [0, 14, 45, 53]
Epoch 00002: early stopping
fininsh
Epoch 00002: early st

Multi-task learning

In [9]:
Avg_mae = np.concatenate([pm_mae,im_mae,em_mae,ts_mae])
#print(Avg_mae)
print('Avg: %.2f ± %.2f' % (sum(Avg_mae)/len(Avg_mae),np.std(Avg_mae)))

Avg: 0.28 ± 0.22


draw model structure

In [10]:
Avg_mae = np.concatenate([pm_mae,im_mae,em_mae])
#print(Avg_mae)
print('Avg: %.2f ± %.2f' % (sum(Avg_mae)/len(Avg_mae),np.std(Avg_mae)))

Avg: 0.21 ± 0.19


training